<a href="https://colab.research.google.com/github/RaiYan163/Transformers_LLM_prac/blob/main/TransformerPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import math

class InputEmbeddings(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.d_model)





In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    #Creating a matrix of shape (seq_len, d_model)

    pe = torch.zeros(seq_len, d_model)

    #Creating a vector of shape (seq_len, 1)
    position = torch.arange(0, seq_len, dtype = torch.float).unsqueeze(1) # Did not understand the purpose of unsqueeze, it is said that it adds a dimension , but how and why?
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/d_model))

    #Applying sin to the even position
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)


    pe = pe.unsqueeze(0) # (1, Seq_len, d_model)
    self.register_buffer('pe', pe) #why we used it? Clear it out

    def forward(self, x):
      x = x + (self.pe[:, :x.shape[1], :]).requires_grad(False) # I am having problems regarding the dimensions
      return self.dropout(x) #please clear out the drop out






In [ ]:
class LayerNormalization(nn.Module):
  def __init__(self, eps: float = 10**-6) -> None:
    super().__init__()
    self.eps = eps #Required for numerical stability
    self.alpha = nn.Parameter(torch.ones(1)) #multipliedm, it is said this makes the param learnable.
    self.bias = nn.Parameter(torch.zeros(1)) #added, it is single tensor (1, )

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim = True) # keeping the single mean in last dimensiion
    std = x.std(dim = -1, keepdim = True)
    return self.alpha * (x-mean) / (std + self.eps) + self.bias #bias and alpha parameter, whole ans er sathe added



In [ ]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model: int, d_ff: int, dropout: float)-> None:
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff) # W1 and B1
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model) # w2 and B2

  def forward(self, x):
    # (Batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
    return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

#why dropout, how and why linear transformation? How relu helping, and what is the purpose of FeedForwardBlock

In [4]:
class MultiHeadAttentionBlock(nn.Module):

  def __init__(self, d_model: int, h: int, dropout: float)-> None:
    super().__init__()
    self.d_model = d_model
    self.h = h
    assert d_model % h == 0, "d_model is not divisble by h"

    self.d_k = d_model // h
    self.w_q = nn.Linear(d_model, d_model) #Wq
    self.w_k = nn.Linear(d_model, d_model) #Wk
    self.w_v = nn.Linear(d_model, d_model) #Wv

    self.w_o = nn.Linear(d_model, d_model) #W0
    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout):
    d_k = query.shape[-1]

    # (Batch, h, Seq_len, d_k) --> (batch, h, seq_len, seq_len)
    attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
      attention_scores.masked_fill(mask == 0, -1e9)
    attention_scores = attention_scores.softmax(dim = -1) # (Batch, h, seq_len, seq_len)
    if dropout is not None:
      attention_scores = dropout(attention_score)

    return (attention_scores @ value), attention_scores


  def forward(self, q, k, v, mask):
    query = self.w_q(q) # (Batch, Seq_Len, d_model) ---> (Batch, Seq_len, d_model)
    key = self.w_k(k) # (Batch, Seq_Len, d_model) ---> (Batch, Seq_len, d_model)
    value = self.w_v(v) # (Batch, Seq_Len, d_model) ---> (Batch, Seq_len, d_model)

    # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
    key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
    value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)
    # Why the transpose was needed, what is it's benifit?

    x, self.attention_score = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

    # (Batch, h, Seq_len, d_k) --> (Batch, Seq_len, h, d_k) --> (Batch, Seq_len, d_model)
    x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

    #(Batch, seq_len, d_model) --> (Batch, Seq_len, d_model)
    return self.w_o(x)

